# HW02: HMM
--by Ruijie Rao

In [1]:
import pandas as pd
import numpy as np
import json

## Data Import

In [2]:
DATA_DIR = 'data/'

In [3]:
train_df = pd.read_csv(DATA_DIR+"train", sep="\t", names=["position_index","word","tag"])
dev_df = pd.read_csv(DATA_DIR+"dev", sep="\t", names=["position_index","word","tag"])
test_df = pd.read_csv(DATA_DIR+"test", sep="\t", names=["position_index","word"])

In [4]:
train_df.head()

,position_index,word,tag
0,1,Pierre,NNP
1,2,Vinken,NNP
2,3,",",","
3,4,61,CD
4,5,years,NNS


## Preprocess

In [5]:
from string import punctuation
def remove_punctuations(df):
    punct_set = list(punctuation)+["''","``"]
    df = df[~df["word"].isin(punct_set)]
    try:
        df = df[~df["tag"].isin(punct_set)]
    except:
        pass
    return df

## Vocabulary

In [6]:
vocab_df = train_df.groupby("word").count()[["tag"]].rename(columns={'tag':'occurences'}).reset_index()

In [7]:
len(vocab_df)

43193

In [11]:
thres = 2
filtered_vocab = vocab_df[vocab_df["occurences"]>=thres]
filtered_vocab = filtered_vocab.append({'word':"< unk >",'occurences':vocab_df[vocab_df["occurences"]<thres]["occurences"].sum()}, ignore_index=True)\
    .sort_values(by="occurences", ascending=False).reset_index(drop=True)

C:\Users\ruiji\AppData\Local\Temp\ipykernel_22208\4132589229.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_vocab = filtered_vocab.append({'word':"< unk >",'occurences':vocab_df[vocab_df["occurences"]<thres]["occurences"].sum()}, ignore_index=True)\


In [12]:
len(filtered_vocab)

23183

In [16]:
filtered_vocab.loc[filtered_vocab["word"]=="< unk >","occurences"]

5    20011
Name: occurences, dtype: int64

In [13]:
filtered_vocab_list = filtered_vocab["word"].to_list()
filtered_vocab_list += ["< -unk >", "< allcap-unk >", "< Title-unk >"]
def pseudo_words_encoding(word, filtered_vocab_list):
    if word in filtered_vocab_list:
        return word
    elif "-" in word:
        return "< -unk >"
    elif word.isupper():
        return "< allcap-unk >"
    elif word[0].isupper():
        return "< Title-unk >"
    else: return "< unk >"

In [14]:
train_df_filtered = train_df.copy()
#train_df_filtered.loc[~train_df_filtered["word"].isin(filtered_vocab_list), "word"] = "< unk >"
train_df_filtered["word"] = train_df["word"].apply(lambda x: pseudo_words_encoding(x,filtered_vocab_list))

In [306]:
filtered_vocab.to_csv(DATA_DIR+"vocab.txt", index=True, sep="\t")

What is the selected threshold for unknown words replacement?
- 5

What is the total size of your vocabulary and what is the total occurrences of the special token ‘< unk >’ after replacement?
- 11688 and 50296


## HMM Model

In [307]:
# Count of States
from collections import Counter
count_states = Counter(train_df_filtered["tag"].values)
count_states["< start >"] = train_df_filtered[train_df_filtered["position_index"]==1]["word"].count()

In [308]:
# Count of s -> s'
transition_vocab = train_df_filtered.rename(columns={'tag':'state'}).copy()
next_state = transition_vocab["state"].copy()
transition_vocab["state'"] = next_state[1:].reset_index(drop=True)
initial_selection = (transition_vocab["position_index"]==1)[1:].reset_index(drop=True)
initial_selection.loc[len(initial_selection)] = True
transition_vocab.loc[initial_selection, "state"] = "< start >"
transition_vocab.loc[len(transition_vocab)-1, "state'"] = transition_vocab.loc[0, "state"]
transition_count = transition_vocab.groupby(["state","state'"])[["word"]].count().reset_index()\
    .rename(columns={"word":"occurences"}).sort_values(by="occurences", ascending=False)

In [309]:
# Transition Prob
def t_prob(x):
        return x.occurences/count_states[x.state]
transition_prob = transition_count
transition_prob["p"] = transition_prob.apply(t_prob, axis=1)
#transition_prob["key"] = transition_prob.apply(lambda x: str((x.state,x["state'"])), axis=1)
transition_prob["key"] = transition_prob.apply(lambda x: (x.state,x["state'"]), axis=1)
transition_prob.set_index("key", inplace=True)
transition_dict = transition_prob[["p"]].to_dict()["p"]

In [310]:
# Count of s -> x
emission_vocab = train_df_filtered.rename(columns={'tag':'state'}).copy()
emission_count = emission_vocab.groupby(["state","word"])[["position_index"]].count().reset_index()\
    .rename(columns={"position_index":"occurences"}).sort_values(by="occurences", ascending=False)

In [311]:
# Emission Prob
emission_prob = emission_count
#emission_prob["key"] = emission_prob.apply(lambda x: str((x.state,x.word)), axis=1)
emission_prob["key"] = emission_prob.apply(lambda x: (x.state,x.word), axis=1)
emission_prob.set_index("key", inplace=True)
emission_prob["p"] = emission_prob.apply(lambda x: x.occurences/count_states[x.state], axis=1)
emission_dict = emission_prob[["p"]].to_dict()["p"]


In [312]:
# Output
'''hmm_model_output = {"transition": transition_dict, "emission": emission_dict}
with open(DATA_DIR+"hmm.json", "w") as file:
    json.dump(hmm_model_output, file)'''

'hmm_model_output = {"transition": transition_dict, "emission": emission_dict}\nwith open(DATA_DIR+"hmm.json", "w") as file:\n    json.dump(hmm_model_output, file)'

In [313]:
# How many transition and emission parameters in your HMM?
display(len(transition_dict))
display(len(emission_dict))

1392

30362

## Greedy Decoding

In [314]:
tag_list = [t for t in count_states.keys() if t != "< start >"]
K  = len(tag_list)
V = len(filtered_vocab_list)
transition_matrix = np.zeros((K,K))
emission_matrix = np.zeros((K,V))
initial_array = np.zeros((1,K))

In [315]:
# transition_matrix & initial_array
for key, value in transition_dict.items():
    state, state_ = key
    j = tag_list.index(state_)
    if state == "< start >":
        initial_array[0,j] = value
        continue
    i = tag_list.index(state)
    transition_matrix[i,j] = value

In [316]:
# emission_matrix
for key, value in emission_dict.items():
    state, word = key
    j = filtered_vocab_list.index(word)
    i = tag_list.index(state)
    emission_matrix[i,j] = value

In [317]:
def greedy_decoding(test_df, initial_array, transition_matrix, emission_matrix, filtered_vocab_list):
    result = []
    for index, row in test_df.iterrows():
        if row.position_index == 1:
            prev_state_index = 0
            t_vec = initial_array
        else:
            t_vec = transition_matrix[prev_state_index,:]
        word_index = filtered_vocab_list.index(pseudo_words_encoding(row.word, filtered_vocab_list))
        e_vec = emission_matrix[:,word_index]
        argmax_index = np.argmax(t_vec*e_vec.T)
        prev_state_index = argmax_index
        argmax_tag = tag_list[argmax_index]
        result.append(argmax_tag)
    return result

In [318]:
dev_pred = greedy_decoding(dev_df, initial_array, transition_matrix, emission_matrix, filtered_vocab_list)

In [319]:
def accuracy(prediction, ground_truth):
    return sum(np.array(prediction) == np.array(ground_truth))/len(ground_truth)

In [320]:
accuracy(dev_pred, dev_df["tag"].values)

0.9412300406775544

In [321]:
dev_df["greedy"] = dev_pred

In [322]:
# Output
test_df["greedy"] = greedy_decoding(test_df, initial_array, transition_matrix, emission_matrix, filtered_vocab_list)
test_df[["position_index","word","greedy"]].to_csv("greedy.out", header=False, index=False, sep="\t")

## Viterbi Algorithm

In [323]:
def create_corpus(words):
    corpus = []
    line = []
    for word in words:
        line.append(word)
        if word==".":
            corpus.append(line)
            line = []
    return corpus
    

In [324]:
class Viterbi:
    def __init__(self, t_dict, e_dict, vocab, tags) -> None:
        self.t_dict= t_dict
        self.e_dict = e_dict
        self.vocab = vocab
        self.tags = tags # without < start >
        self.K = len(self.tags)

    def predict(self, corpus:list[str]):
        N = len(corpus)
        M = np.zeros((N, self.K)) # Num of words x Num of tags
        states_record = np.zeros((N-1, self.K)) # Records the best previous states except for the initial
        for i, word_ in enumerate(corpus):
            word = pseudo_words_encoding(word_, self.vocab)
            for j in range(self.K):
                tag = self.tags[j]
                if i == 0: # initial state
                    M[i,j] = self.t_dict.get(('< start >', tag),0) * self.e_dict.get((tag, word),0)
                else:
                    temp = [M[i-1,k]*self.t_dict.get((tag_,tag),0)*self.e_dict.get((tag, word),0) for k,tag_ in enumerate(self.tags)]
                    best_prev_state_i = np.argmax(temp)
                    states_record[i-1,j] = best_prev_state_i
                    M[i,j] = temp[best_prev_state_i]
        # Backward
        result = []
        best_state_i = np.argmax(M[-1,:])
        result.append(self.tags[best_state_i])
        for i in range(len(states_record)-1,-1,-1):
            best_state_i = int(states_record[i,best_state_i])
            result.append(self.tags[best_state_i])
        return result[::-1]

In [325]:
myViterbi = Viterbi(transition_dict, emission_dict, filtered_vocab_list, tag_list)

In [326]:
dev_pred = []
for sentence in create_corpus(dev_df["word"].values):
    dev_pred += myViterbi.predict(sentence)
accuracy(dev_pred, dev_df["tag"].values)

0.9498664319106308

In [327]:
result = []
for sentence in create_corpus(test_df["word"].values):
    result += myViterbi.predict(sentence)

In [328]:
# Output
test_df["viterbi"] = result
test_df[["position_index","word","viterbi"]].to_csv("viterbi.out", header=False, index=False, sep="\t")

# Evaluation

In [329]:
dev_df["viterbi"] = dev_pred

### Greedy

In [330]:
dev_df["filtered_word"] = dev_df["word"]
dev_df.loc[~dev_df["word"].isin(filtered_vocab_list), "filtered_word"] = "< unk >"

In [331]:
error_set = dev_df[dev_df["tag"]!=dev_df["greedy"]][["position_index","filtered_word","word","tag","greedy"]]

In [332]:
error_set.groupby(["tag","greedy"]).count()["word"].sort_values(ascending=False).head(20)

tag   greedy
NN    JJ        531
JJ    NN        394
NN    NNP       346
VBN   VBD       313
VBD   VBN       270
NNS   NN        263
WDT   IN        248
RB    IN        220
NNP   JJ        213
JJ    NNP       200
NN    CD        184
IN    RB        151
JJ    VBN       145
      CD        141
NNS   CD        139
      NNP       139
VBP   VB        138
RB    JJ        134
NNPS  NNP       127
VBG   NN        127
Name: word, dtype: int64

In [333]:
error_set[(error_set["tag"]=="JJ") & (error_set["greedy"]=="NN")].head(30)

,position_index,filtered_word,word,tag,greedy
771,29,editorial,editorial,JJ,NN
813,7,< unk >,irreverent,JJ,NN
824,18,close,close,JJ,NN
828,22,editorial,editorial,JJ,NN
875,17,< unk >,bona,JJ,NN
876,18,< unk >,fide,JJ,NN
1109,10,editorial,editorial,JJ,NN
1327,6,< unk >,shoddy,JJ,NN
1328,7,editorial,editorial,JJ,NN
1720,22,bulk,bulk,JJ,NN


In [334]:
emission_dict[("NN","< -unk >")]

0.005253501027177067

In [335]:
unk_set = dev_df[dev_df["filtered_word"]=='< unk >']

In [336]:
accuracy(unk_set["greedy"],unk_set["tag"])

0.6050286441756842

In [337]:
transition_dict[("< start >","NNP")]

0.19789104610393007